<a href="https://colab.research.google.com/github/waterta0115/LDA_graduattion_thesis/blob/main/network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def find_threshold_by_count(matrix, target_edge_count):

    indices = np.triu_indices(len(matrix), k=1)
    edge_weights = matrix[indices]

    existing_edges = edge_weights[edge_weights > 0]

    if len(existing_edges) == 0:
        print("Error: No edges exist in the graph.")
        return 0.0

    # --- 2. Threshold Calculation Logic ---
    sorted_weights = np.sort(existing_edges)
    total_edges = len(existing_edges)

    if target_edge_count >= total_edges:
        print("Warning: The specified number of edges is greater than or equal to the total number of edges. All edges will be kept.")
        tau = 0.0 # To keep all edges, set to 0 (or a value smaller than the minimum)

        calculated_cut_percentage = 0.0
    elif target_edge_count <= 0:
        print("Warning: The specified number of edges is 0 or less. All edges will be removed.")
        tau = sorted_weights[-1] + 1.0 # Set higher than the maximum value
        calculated_cut_percentage = 100.0
    else:
        cutoff_index = total_edges - target_edge_count - 1

        # Determine the threshold (typically, values greater than this are kept)
        tau = sorted_weights[cutoff_index]

        calculated_cut_percentage = (cutoff_index + 1) / total_edges * 100

    # --- 3. Verification of Results ---
    actual_kept_edges = np.sum(existing_edges > tau)

    print(f"--- Threshold Calculation Result (Target Count Mode) ---")
    print(f"Total edges (greater than 0) : {total_edges}")
    print(f"Target number of edges to keep : {target_edge_count}")
    print(f"Determined threshold (τ)    : {tau:.7f}")
    print(f"---")
    print(f"Actual edges kept    : {actual_kept_edges} (those greater than threshold {tau:.4f})")
    print(f"Actual percentage kept        : {actual_kept_edges / total_edges * 100:.2f}%")
    print(f"Calculated cut percentage      : {calculated_cut_percentage:.2f}%")

    if actual_kept_edges != target_edge_count:
        print(f"*Note: Due to edges with identical weights, the target count ({target_edge_count}) was not perfectly matched.")

    # --- 4. Create and Visualize the Cumulative Probability Plot (CDF) ---
    y_cumulative = np.arange(1, len(sorted_weights) + 1) / len(sorted_weights)

    plt.figure(figsize=(12, 7))
    plt.plot(sorted_weights, y_cumulative, color='blue', label='Cumulative Distribution (CDF)')

    # --- 5. Display the Determined Threshold on the Graph ---
    plt.axvline(x=tau, color='red', linestyle='--',
                label=f'Threshold τ = {tau:.7f}')

    plt.axhline(y=calculated_cut_percentage / 100.0, color='red', linestyle=':',
                label=f'Cutoff Probability {calculated_cut_percentage/100:.2f} (Removes bottom {calculated_cut_percentage:.1f}%)')

    # Mark the intersection point
    plt.plot(tau, calculated_cut_percentage / 100.0, 'ro')

    plt.title(f'CDF of Edge Weights (Targeting top {target_edge_count} edges)',fontsize=20)
    plt.xlabel('Edge Weight (Value of element)',fontsize=20)
    plt.ylabel('Cumulative Probability',fontsize=20)
    plt.legend(fontsize=20)
    plt.grid(True, which="both", ls="--", alpha=0.5)

    plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

    plt.show()

    return tau

In [ ]:
def adj_matrix_(topic_term_dists,vocab,threshold):
    co_occurrence = np.dot(topic_term_dists.T, topic_term_dists)
    co_matrix = pd.DataFrame(co_occurrence, index=vocab, columns=vocab)
    threshold = np.quantile(co_matrix.values, threshold)
    filtered = co_matrix.mask(co_matrix < threshold, 0)
    adj_matrix = filtered.to_numpy(dtype=float)
    return adj_matrix

In [2]:
def plot_igraph_from_adjacency_deg(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # Convert adjacency matrix to numpy array
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # Set edge weights below the threshold to 0
    adj = np.where(adj > edge_threshold, adj, 0)

    # Create igraph graph
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # Remove isolated nodes
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # Node labels
    # Assuming labels = vocab is passed
    if "name" in g.vs.attributes():
        # Copy from g.vs["name"] to g.vs["label"]
        g.vs["label"] = g.vs["name"]
    else:
        # Fallback if "name" attribute is not present
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # Assign labels only to the top_n_labels nodes with the highest degrees
    degrees = g.degree()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # Top top_n_labels items
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # Layout
    layout = g.layout(layout_type)

    # Edge width
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # Community detection (e.g., Louvain method, fallback to fastgreedy if Louvain not available)
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # Change node size according to node degree
    degrees = g.degree()
    # Specify minimum and maximum sizes for appropriate scaling
    min_size = 20
    max_size = 80
    # Scaling
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # Calculate node size
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # Redraw based on node size (node size = degree)
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

In [3]:
def plot_igraph_from_adjacency_bet(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # Convert adjacency matrix to numpy array
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # Set edge weights below the threshold to 0
    adj = np.where(adj > edge_threshold, adj, 0)

    # Create igraph graph
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # Remove isolated nodes
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # Node labels
    # Assuming labels = vocab is passed
    if "name" in g.vs.attributes():
        # Copy from g.vs["name"] to g.vs["label"]
        g.vs["label"] = g.vs["name"]
    else:
        # Fallback if "name" attribute is not present
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # Assign labels only to the top_n_labels nodes with the highest betweenness
    degrees = g.betweenness()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # Top top_n_labels items
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # Layout
    layout = g.layout(layout_type)

    # Edge width
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # Community detection (e.g., Louvain method, fallback to fastgreedy if Louvain not available)
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # Change node size according to node betweenness
    degrees = g.betweenness()
    # Specify minimum and maximum sizes for appropriate scaling
    min_size = 20
    max_size = 80
    # Scaling
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # Calculate node size
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # Redraw based on node size (node size = betweenness)
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

In [4]:
def plot_igraph_from_adjacency_pr(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # Convert adjacency matrix to numpy array
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # Set edge weights below the threshold to 0
    adj = np.where(adj > edge_threshold, adj, 0)

    # Create igraph graph
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # Remove isolated nodes
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # Node labels
    # Assuming labels = vocab is passed
    if "name" in g.vs.attributes():
        # Copy from g.vs["name"] to g.vs["label"]
        g.vs["label"] = g.vs["name"]
    else:
        # Fallback if "name" attribute is not present
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # Assign labels only to the top_n_labels nodes with the highest PageRank
    degrees = g.pagerank()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # Top top_n_labels items
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # Layout
    layout = g.layout(layout_type)

    # Edge width
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # Community detection (e.g., Louvain method, fallback to fastgreedy if Louvain not available)
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # Change node size according to node PageRank
    degrees = g.pagerank()
    # Specify minimum and maximum sizes for appropriate scaling
    min_size = 20
    max_size = 80
    # Scaling
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # Calculate node size
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # Redraw based on node size (node size = pagerank)
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

# 評価指標

In [9]:
import pandas as pd
import numpy as np
import igraph as ig
from scipy import sparse

def get_node_metrics_fast(graph, communities):
    """
    High-speed version using matrix operations:
    Calculates participation coefficient, z-score, betweenness, and PageRank for all nodes,
    and returns a DataFrame containing all results (no filtering).
    """
    # Get community information
    membership = np.array(communities.membership)
    num_communities = len(communities)
    num_nodes = graph.vcount()

    # --- 1. Preparation for Speedup: Create Sparse Matrix ---
    A = graph.get_adjacency_sparse()

    # U: Affiliation matrix
    row_indices = np.arange(num_nodes)
    col_indices = membership
    data = np.ones(num_nodes)
    U = sparse.csr_matrix((data, (row_indices, col_indices)), shape=(num_nodes, num_communities))

    # K_dist: Degree matrix per community
    K_dist = A @ U

    # k_i: Total degree of each node
    k_i = np.array(A.sum(axis=1)).flatten()

    # --- A. Calculate Participation Coefficient ---
    k_i_safe = np.where(k_i == 0, 1, k_i)
    K_dist_sq = K_dist.power(2)
    sum_k_is_sq = np.array(K_dist_sq.sum(axis=1)).flatten()
    participation_coeffs = 1.0 - (sum_k_is_sq / (k_i_safe ** 2))
    participation_coeffs[k_i == 0] = 0.0

    # --- B. Calculate Community-internal Degree (z-score) ---
    k_in = np.array(K_dist[np.arange(num_nodes), membership]).flatten()

    df_temp = pd.DataFrame({
        'comm_id': membership,
        'k_in': k_in
    })

    grouped = df_temp.groupby('comm_id')['k_in']
    means = grouped.transform('mean')
    stds = grouped.transform('std')
    stds = stds.replace(0, 1)

    z_scores = (df_temp['k_in'] - means) / stds
    z_scores = z_scores.fillna(0).values

    # --- C. Consolidate Data ---

    # Calculate PageRank, etc.
    degree = k_i
    betweenness = graph.betweenness()
    pagerank = graph.pagerank()

    # Create DataFrame including all nodes
    df = pd.DataFrame({
        "name": graph.vs["name"],
        "community_id": membership, # This will contain Louvain results (0, 1, 2...)
        "degree": degree,
        "pagerank": pagerank,
        "betweenness": betweenness,
        "z_score": z_scores,
        "participation": participation_coeffs
    })

    # No filtering is done here; all data is returned
    return df


def get_top10_per_community(graph, communities):
    """
    1. Calculate metrics for all nodes at once.
    2. Extract the top 10 nodes per community, ordered by importance (here, degree).
    """

    # 1. Calculate for all nodes (no loops are used)
    all_nodes_df = get_node_metrics_fast(graph, communities)

    # 2. Use Pandas functionality to extract "top 10 per community"
    #    If you want to change the sorting order, change to by="pagerank", etc.
    final_df = (
        all_nodes_df
        .sort_values(["community_id", "degree"], ascending=[True, False])
        .groupby("community_id")
        .head(10)  # Get the top 10 items for each group
    )

    return final_df,all_nodes_df

ModuleNotFoundError: No module named 'igraph'

In [6]:
def plot_network_scatter(df,
                         x_col="degree",
                         y_col="betweenness",
                         hue_col="community_id",
                         size_col="pagerank",
                         name_col="name",        # Added: Column name containing node names
                         label_threshold_x=None,
                         figsize=(12, 8)):
    """
    Plots a scatter chart of network metrics and labels points using values from the 'name' column.
    """

    # Prepare the figure
    plt.figure(figsize=figsize)

    # If community_id, etc. are numerical, convert to string to treat as categorical
    plot_df = df.copy()
    if hue_col in plot_df.columns:
        plot_df[hue_col] = plot_df[hue_col].astype(str)

    # Draw the scatter plot
    sns.scatterplot(
        data=plot_df,
        x=x_col,
        y=y_col,
        hue=hue_col,
        size=size_col,
        sizes=(50, 600),
        alpha=0.7,
        palette="tab10",
        edgecolor="black"
    )

    # --- Labeling process (modified part) ---
    if label_threshold_x is not None:
        # Extract only rows that satisfy the threshold condition
        labels_df = df[df[y_col] >= label_threshold_x]

        # Loop through each row (get info from row, not just index)
        for _, row in labels_df.iterrows():
            plt.text(
                x=row[x_col],
                y=row[y_col],
                s=row[name_col],  # Modified: Use the value from the specified column (name)
                fontsize=13,
                color='black',
                fontweight='bold',
                ha='right',
                va='bottom'
            )

    # Set grid and title
    plt.title(f"Network Metrics: {x_col} vs {y_col}", fontsize=20)
    plt.xlabel(x_col, fontsize=20)
    plt.ylabel(y_col, fontsize=20)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Move legend outside the plot area
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0,fontsize=20)

    plt.tight_layout()
    plt.show()

In [7]:
def hist_dist(df):
    plt.figure(figsize=(8, 6))
    # Plot histogram
    plt.hist(df['degree'], bins=50, log=True, color='skyblue', edgecolor='black')
    plt.xlabel("Degree (log scale)",fontsize=20) # Plot axis itself with log transform if necessary
    plt.ylabel("Frequency (log scale)",fontsize=20)
    plt.title("Degree Distribution (Log-Log Scale)",fontsize=20)
    plt.show()

In [8]:
def plot_community_violin(df,
                          x_col="community_id",
                          y_col="pagerank",
                          figsize=(12, 6),
                          fontsize_title=20,
                          fontsize_label=20,
                          fontsize_tick=20):
    # Set figure size
    plt.figure(figsize=figsize)

    plot_df = df.copy()
    if x_col in plot_df.columns:
        plot_df[x_col] = plot_df[x_col].astype(str)

    sns.violinplot(
        data=plot_df,
        x=x_col,
        y=y_col,
        palette="Set3",
        inner=None,
        linewidth=1
    )

    # 2. Stripplot (scatter actual data points)
    # Overlaying this allows for intuitive understanding of data density and count.
    sns.stripplot(
        data=plot_df,
        x=x_col,
        y=y_col,
        color='black',
        alpha=0.3, # Transparency of points
        size=3     # Size of points
    )
    plt.title(f"Distribution of {y_col} by {x_col}", fontsize=fontsize_title)

    # Font size for X-axis and Y-axis labels (e.g., "community_id")
    plt.xlabel(x_col, fontsize=fontsize_label)
    plt.ylabel(y_col, fontsize=fontsize_label)

    # Font size for ticks (numbers or category names on axes)
    plt.tick_params(axis='both', which='major', labelsize=fontsize_tick)
    plt.tight_layout()
    plt.show()